In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import joblib
import os

############################################################
# Step 1: Load the Data and Model
############################################################

# File paths
X_test_path = r'data\combined\X_test.csv'
y_test_path = r'data\combined\y_test.csv'
model_path = r'data\models\route_predictor_model.pkl'
player_play_path = r'data\combined\final_player_play_data.csv'
players_path = r'data\raw\players.csv'

# Load test data and model
X_test = pd.read_csv(X_test_path)
y_test = pd.read_csv(y_test_path)
player_play_df = pd.read_csv(player_play_path)
players_df = pd.read_csv(players_path)

# Load the trained XGBoost model
xgb_model = joblib.load(model_path)

# Predict probabilities for each play
print("Predicting probabilities for each route on the test set...")
route_probs = xgb_model.predict_proba(X_test)

############################################################
# Step 2: Calculate Route Deceptiveness Score (RDS)
############################################################

# Add route probabilities and actual routes to a DataFrame
results_df = X_test.copy()
results_df['actual_route'] = y_test
results_df['predicted_route'] = np.argmax(route_probs, axis=1)
results_df['max_prob'] = np.max(route_probs, axis=1)
results_df['actual_prob'] = [route_probs[i, y_test.iloc[i, 0]] for i in range(len(y_test))]

# Calculate deceptiveness score for each play
results_df['deceptiveness_score'] = 1 - (results_df['actual_prob'] / results_df['max_prob'])

############################################################
# Step 3: Aggregate Deceptiveness Scores by Wide Receiver
############################################################

# Merge player information into results to identify which player was involved in each play
results_df['gameId'] = player_play_df['gameId']
results_df['playId'] = player_play_df['playId']
results_df['nflId'] = player_play_df['nflId']

# Merge player position data to identify only wide receivers
players_df['nflId'] = players_df['nflId'].astype(float)
results_df = results_df.merge(players_df[['nflId', 'displayName', 'position']], on='nflId', how='left')

# Filter to only include wide receivers
wr_results_df = results_df[results_df['position'] == 'WR']

# Calculate average Route Deceptiveness Score (RDS) for each wide receiver
wr_rds = wr_results_df.groupby(['nflId', 'displayName'])['deceptiveness_score'].mean().reset_index()
wr_rds.rename(columns={'deceptiveness_score': 'route_deceptiveness_score'}, inplace=True)

# Rank wide receivers by their Route Deceptiveness Score
wr_rds['rank'] = wr_rds['route_deceptiveness_score'].rank(ascending=False)

# Sort by RDS for display
wr_rds = wr_rds.sort_values(by='route_deceptiveness_score', ascending=False)

# Display the top 10 most deceptive wide receivers
print("\nTop 10 Most Deceptive Wide Receivers by Route Deceptiveness Score (RDS):")
print(wr_rds.head(10))

############################################################
# Step 4: Save the RDS results to a CSV (Including nflId)
############################################################

# Save RDS for each WR to a CSV file, now including nflId
output_path = r'data\combined\route_deceptiveness_scores.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
wr_rds.to_csv(output_path, index=False)
print(f"Route Deceptiveness Scores saved to {output_path}")


Predicting probabilities for each route on the test set...

Top 10 Most Deceptive Wide Receivers by Route Deceptiveness Score (RDS):
       nflId       displayName  route_deceptiveness_score  rank
209  54850.0     Kyric McGowan                   0.508833   1.0
170  53538.0   Dez Fitzpatrick                   0.500000   2.0
18   41500.0      Willie Snead                   0.437544   3.0
9    40024.0      Keenan Allen                   0.384976   4.0
27   42437.0     Ty Montgomery                   0.375511   5.0
120  48988.0  Gunner Olszewski                   0.370701   6.0
51   44908.0    Kenny Golladay                   0.369035   7.0
40   43495.0       Mike Thomas                   0.367018   8.0
115  48576.0       Jason Moore                   0.366171   9.0
145  52620.0     Dezmon Patmon                   0.360339  10.0
Route Deceptiveness Scores saved to data\combined\route_deceptiveness_scores.csv


In [5]:
import pandas as pd

# File paths
player_play_path = r'data\combined\final_player_play_data.csv'
route_deceptiveness_scores_path = r'data\combined\route_deceptiveness_scores.csv'

# Load player play data and route deceptiveness scores
player_play_df = pd.read_csv(player_play_path)
route_deceptiveness_scores_df = pd.read_csv(route_deceptiveness_scores_path)

# Count the number of routes run for each wide receiver
route_counts = player_play_df.groupby('nflId')['routeRan'].count().reset_index()
route_counts.rename(columns={'routeRan': 'route_count'}, inplace=True)

# Merge the route counts with the route deceptiveness scores
combined_df = route_deceptiveness_scores_df.merge(route_counts, on='nflId', how='left')

# Filter wide receivers who have more than 100 routes run
filtered_df = combined_df[combined_df['route_count'] > 100]

# Sort by route deceptiveness score and select the top 10
top_10_deceptive_wrs = filtered_df.sort_values(by='route_deceptiveness_score', ascending=False).head(10)

# Display the top 10 most deceptive wide receivers with more than 100 routes run
print("\nTop 10 Most Deceptive Wide Receivers with More Than 100 Routes Run:")
print(top_10_deceptive_wrs)



Top 10 Most Deceptive Wide Receivers with More Than 100 Routes Run:
      nflId       displayName  route_deceptiveness_score  rank  route_count
11  52500.0    Devin Duvernay                   0.341668  12.0          184
18  47954.0    Darius Slayton                   0.331973  19.0          106
20  52454.0       K.J. Hamler                   0.328276  21.0          140
22  47847.0        DK Metcalf                   0.326276  23.0          218
25  46150.0    Michael Gallup                   0.324164  26.0          114
29  52430.0  Justin Jefferson                   0.319648  30.0          302
31  39989.0      Robert Woods                   0.317917  32.0          148
34  44003.0    Marcus Johnson                   0.315577  35.0          109
35  54476.0       Chris Olave                   0.315339  36.0          246
36  48489.0       David Sills                   0.315275  37.0          102


In [19]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

# File paths
player_data_path = r"C:\Users\RaymondCarpenter\Documents\GitHub\bigdatabowl2025\data\combined\final_player_play_data.csv"
route_deceptiveness_path = r"C:\Users\RaymondCarpenter\Documents\GitHub\bigdatabowl2025\data\combined\route_deceptiveness_scores.csv"

# Load player-route data
df_player_play = pd.read_csv(player_data_path)
df_route_deceptiveness = pd.read_csv(route_deceptiveness_path)

# Convert relevant fields to appropriate types
df_player_play['gameId'] = df_player_play['gameId'].astype(str)
df_player_play['playId'] = df_player_play['playId'].astype(int)
df_player_play['nflId'] = df_player_play['nflId'].astype(float)

# Aggregate total routes by player
df_summary = df_player_play.groupby('nflId').agg(
    total_routes=('routeRan', 'count')
).reset_index()

# Merge with route deceptiveness scores
df_merged = pd.merge(
    df_summary,
    df_route_deceptiveness[['nflId', 'displayName', 'route_deceptiveness_score']],
    on='nflId',
    how='left'
)

# Filter players with more than 100 routes run
filtered_players = df_merged[df_merged['total_routes'] > 100]

# Rank by route deceptiveness score and select top 10
top_10_filtered = filtered_players.sort_values('route_deceptiveness_score', ascending=False).head(10)

# Select relevant columns for display
columns_to_display = [
    'displayName', 'route_deceptiveness_score', 'total_routes'
]

# Adding additional columns that might be useful (if available)
if 'team' in df_merged.columns:
    columns_to_display.append('team')
if 'position' in df_merged.columns:
    columns_to_display.append('position')

# Create the final DataFrame for the top 10 filtered players
top_10_filtered_display = top_10_filtered[columns_to_display]

# Helper functions for styling
def value_to_cell_color(value, ranges):
    if (ranges[1] - ranges[0]) == 0:
        norm_value = 0.5
    else:
        norm_value = (value - ranges[0]) / (ranges[1] - ranges[0])
    cmap = plt.get_cmap('summer_r')
    norm = mcolors.Normalize(vmin=0, vmax=1)
    scalar_mappable = cm.ScalarMappable(cmap=cmap, norm=norm)
    rgba_color = scalar_mappable.to_rgba(norm_value)
    hex_color = mcolors.to_hex(rgba_color)
    return hex_color

def value_to_text_color(value, ranges):
    if (ranges[1] - ranges[0]) == 0:
        norm_value = 0.5
    else:
        norm_value = (value - ranges[0]) / (ranges[1] - ranges[0])
    return "black" if norm_value < 0.5 else "white"

def make_pretty(styler):
    styler.set_caption("Top 10 Players with More Than 100 Routes").set_table_styles(
        [{'selector': 'caption', 'props': 'caption-side: bottom; font-size:1.25em;'}],
        overwrite=False
    )
    styler.format(precision=2)
    return styler

def color_rates(styler, df):
    v_routes = [df["total_routes"].min(), df["total_routes"].max()]
    v_rds = [df["route_deceptiveness_score"].min(), df["route_deceptiveness_score"].max()]
    styler.map(lambda v: f'background: {value_to_cell_color(v, v_routes)}; color: {value_to_text_color(v, v_routes)}', subset=["total_routes"])
    styler.map(lambda v: f'background: {value_to_cell_color(v, v_rds)}; color: {value_to_text_color(v, v_rds)}', subset=["route_deceptiveness_score"])
    return styler

# Apply styling to the `top_10_filtered_display` DataFrame
styled_df = top_10_filtered_display.style.pipe(make_pretty)
styled_df = color_rates(styled_df, top_10_filtered)

# Export to HTML file
html_table = styled_df.to_html()

# Save to file (optional)
with open("top_10_filtered_table.html", "w") as f:
    f.write(html_table)

# Display HTML in Jupyter Notebook
display(styled_df)


,displayName,route_deceptiveness_score,total_routes
293,Devin Duvernay,0.34,184
233,Darius Slayton,0.33,106
280,K.J. Hamler,0.33,140
216,DK Metcalf,0.33,218
160,Michael Gallup,0.32,114
271,Justin Jefferson,0.32,302
16,Robert Woods,0.32,148
94,Marcus Johnson,0.32,109
407,Chris Olave,0.32,246
253,David Sills,0.32,102
